In [2]:
import numpy as np
import pandas as pd
from scipy import stats

%precision 3
np.random.seed(1111)

In [3]:
df = pd.read_csv('/Users/yuya/Desktop/Python_Static/python_stat_sample/data/ch11_potato.csv')
sample = np.array(df['重さ'])
sample

array([122.02, 131.73, 130.6 , 131.82, 132.05, 126.12, 124.43, 132.89,
       122.79, 129.95, 126.14, 134.45, 127.64, 125.68])

In [4]:
# 標本平均を求める
s_mean = np.mean(sample)
s_mean

128.4507142857143

In [5]:
# 母平均は130gという仮定で仮説検定を行う
# ※フライドポテトの母集団は正規分布で、その分散は9とする

# 標本平均XがP(X<=x) = 0.05 を満たすxを考える
rv = stats.norm(130, np.sqrt(9/14))
rv.isf(0.95)

128.68118313069039

In [6]:
# 上記の計算により、標本平均が128.68g以下になる確率は
# 5%である事が分かった
# Aさんの抽出した標本平均が128.451gなので、
# これは5%しか起きない珍しい出来事だと考えられる

# 標準化した値で再計算してみる
# 検定統計量
z = (s_mean - 130) / np.sqrt(9/14)
z

-1.932298779026813

In [7]:
# 臨界値を求める
rv = stats.norm()
rv.isf(0.95)

-1.6448536269514722

In [8]:
# 検定統計量 < 臨界値
# だったので、帰無仮説は棄却され、「平均は130gより少ない」という結論になる

# p値を使った仮説検定
# p値を求める
rv.cdf(z)

0.026661319523126635

In [9]:
# 130gより多い・少ない場合の両方を考える
# →両側検定

# フライドポテトの例で有意水準5%の両側検定を行う
z = (s_mean - 130) / np.sqrt(9/14)
z

-1.932298779026813

In [10]:
rv = stats.norm()
rv.interval(0.95)

(-1.959963984540054, 1.959963984540054)

In [11]:
rv.cdf(z) * 2

0.05332263904625327

In [12]:
# 第一種の過誤→帰無仮説が正しいときに、帰無仮説を棄却してしまう過誤
# 第二種の過誤→対立仮説が正しいときに、帰無仮説を採択してしまう過誤

# 今回の例でいうと
# 第一種の過誤→「平均が130g」であるにも関わらず、「平均は130gより少ない」
rv = stats.norm(130, 3)

In [13]:
# 第一種の過誤
# この母集団から14個の標本を抽出して仮説検定を行うという
# 作業を10000回行い、「平均が130g」であるのにも関わらず
# 「平均は130gより少ない」と結論づけてしまう割合を算出
c = stats.norm().isf(0.95)
n_samples = 10000
cnt = 0

for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14), 2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130) / np.sqrt(9/14)
    if z < c:
        cnt += 1
        
cnt / n_samples

# 第一種の過誤を犯す確率を危険率と呼ぶ
# 危険率は有意水準と一致するので、分析のパラメータ決め次第で
# 制御する事ができる

0.053

In [16]:
# 第二種の過誤
# 実際は「母平均が130gより少ない」にも関わらず「母平均は130gより少ない」
# という結論を得ることができない状況
# →本来検出すべき物を検出できていないので、「検出漏れ」とも呼ぶ
rv = stats.norm(128, 3)
# このケースになる割合を求める
c = stats.norm().isf(0.95)
n_samples = 10000
cnt = 0
for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14), 2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130) / np.sqrt(9/14)
    if z >= c:
        cnt += 1
cnt / n_samples

# 第二種の過誤を犯す確率には記号βが使われ、1-βを検出力という。
# このβは母集団の情報に依存するため、分析者が制御できない。

0.190

In [18]:
# 正規分布の母平均の仮説検定を実装
# →母平均がある値μではないと主張するための検定
def pmean_test(sample, mean0, p_var, alpha=0.05):
    s_mean = np.mean(sample)
    n = len(sample)
    rv = stats.norm()
    interval = rv.interval(1-alpha)
    
    z = (s_mean - mean0) / np.sqrt(p_var/n)
    if interval[0] <= z <= interval[1]:
        print('帰無仮説を採択')
    else:
        print('帰無仮説を棄却')
    if z < 0:
        p = rv.cdf(z) * 2
    else:
        p = (1 - rv.cdf(z)) * 2
    print(f'p値は {p: .3f}')

In [19]:
pmean_test(sample, 130, 9)

帰無仮説を採択
p値は  0.053


In [20]:
# 正規分布の母分散の仮説検定を実装
# →母平均がある値σ^2ではないと主張するための検定
def pvar_test(sample, var0, alpha=0.05):
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.chi2(df=n-1)
    interval = rv.interval(1-alpha)
    
    y = (n-1) * u_var / var0
    if interval[0] <= y <= interval[1]:
        print('帰無仮説を採択')
    else:
        print('帰無仮説を棄却')
    if y < rv.isf(0.5):
        p = rv.cdf(y) * 2
    else:
        p = (1 - rv.cdf(y)) * 2
    print(f'p値は{p: .3f}')

In [21]:
pvar_test(sample, 9)

帰無仮説を採択
p値は 0.085


In [30]:
# 正規分布の母平均の仮説検定を実装
# →母分散がわからない状況でのt検定

def pmean_test(sample, mean0, alpha=0.05):
    s_mean = np.mean(sample)
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.t(df=n-1)
    interval = rv.interval(1-alpha)
    
    t = (s_mean - mean0) / np.sqrt(u_var / n)
    if interval[0] <= t <= interval[1]:
        print('帰無仮説を採択')
    else:
        print('帰無仮説を棄却')
    if t < 0:
        p = rv.cdf(t) * 2
    else:
        p = (1 - rv.cdf(t)) * 2
    print(f'p値は{p: .3f}')

In [31]:
pmean_test(sample, 130)

帰無仮説を採択
p値は 0.169


In [32]:
t, p = stats.ttest_1samp(sample, 130)
t, p

(-1.4551960206404198, 0.16933464230414275)